# AI Config Variations - Cookbook

This cookbook contains all code from `aiconfig-variations/SKILL.md` for managing AI Config variations.

## Prerequisites
- `LAUNCHDARKLY_API_TOKEN`: API token with `ai-configs:write` permission
- Project created (run `cookbook_aiconfig_projects.ipynb` first)
- Tools created (run `cookbook_aiconfig_tools.ipynb` first)
- AI Configs created (run `cookbook_aiconfig_create.ipynb` first)

In [1]:
%pip install requests python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load environment variables from .env
import os
from dotenv import load_dotenv

def find_repo_root():
    """Find repo root by looking for .git directory"""
    path = os.getcwd()
    while path != "/":
        if os.path.exists(os.path.join(path, ".git")):
            return path
        path = os.path.dirname(path)
    return os.getcwd()

REPO_ROOT = find_repo_root()
ENV_FILE = os.path.join(REPO_ROOT, ".env")
load_dotenv(ENV_FILE)
print(f"Loaded environment from: {ENV_FILE}")

Loaded environment from: /Users/ld_scarlett/Documents/Github/agent-skills/.env


In [3]:
import requests
import os
import time

API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")
PROJECT_KEY = "support-ai"

# Helper to clean up variations before tests
def delete_variation(config_key: str, variation_key: str):
    """Delete a variation from an AI Config."""
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{config_key}/variations/{variation_key}"
    headers = {"Authorization": API_TOKEN}
    response = requests.delete(url, headers=headers)
    if response.status_code == 204:
        print(f"[OK] Deleted variation '{variation_key}'")
        return True
    elif response.status_code == 404:
        print(f"[INFO] Variation '{variation_key}' does not exist")
        return False
    else:
        print(f"[ERROR] Failed to delete variation: {response.text}")
        return False

---
## 1. Add Agent Mode Variations
From: `SKILL.md` lines 32-132

**Note:** Tools (`search_knowledge_base`, `get_customer_info`) must be created first via `cookbook_aiconfig_tools.ipynb`

In [ ]:
def add_agent_variations(config_key: str):
    """Add variations to an existing agent mode AI Config."""
    variations = [
        {
            "key": "base-config",
            "name": "Base Configuration",
            "instructions": """You are a helpful customer support agent.

Your responsibilities:
- Answer customer questions
- Resolve issues efficiently
- Maintain a friendly tone

Company: {{company_name}}
Priority: {{support_priority}}""",
            "messages": [],  # Required for agent mode
            "model": {
                "modelName": "gpt-4",
                "parameters": {
                    "temperature": 0.7,
                    "maxTokens": 2000,
                    # Optional: custom parameters for your application
                    "custom_param": "value",
                    "enable_feature": True
                }
            },
            # Optional: tools created via aiconfig-tools skill
            "tools": [
                {"key": "search_knowledge_base", "version": 1},
                {"key": "get_customer_info", "version": 1}
            ]
        },
        {
            "key": "advanced-config",
            "name": "Advanced Configuration",
            "instructions": """You are an expert customer support specialist.

Provide detailed assistance with:
- Complex technical issues
- Account management
- Product recommendations

Company: {{company_name}}
Priority: {{support_priority}}""",
            "messages": [],  # Required for agent mode
            "model": {
                "modelName": "gpt-4-turbo",
                "parameters": {
                    "temperature": 0.5,
                    "maxTokens": 4000,
                    "reasoning_depth": "deep"
                }
            },
            "tools": [
                {"key": "search_knowledge_base", "version": 1},
                {"key": "get_customer_info", "version": 1}
            ]
        }
    ]

    # Add each variation
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{config_key}/variations"
    headers = {
        "Authorization": API_TOKEN,
        "Content-Type": "application/json",
        "LD-API-Version": "beta"
    }

    created_variations = []

    for variation in variations:
        response = requests.post(url, headers=headers, json=variation)

        if response.status_code in [200, 201]:
            print(f"  [OK] Added variation: {variation['key']}")
            created_variations.append(variation['key'])
            time.sleep(0.5)
        else:
            print(f"  [ERROR] Failed to add variation {variation['key']}: {response.text}")

    if created_variations:
        print(f"\n[OK] Added {len(created_variations)} variations to '{config_key}'")
        print(f"  URL: https://app.launchdarkly.com/projects/{PROJECT_KEY}/ai-configs/{config_key}")
        return True

    return False

# Clean up first, then add variations
print("=== Adding Agent Mode Variations to 'support-agent' ===\n")
delete_variation("support-agent", "base-config")
delete_variation("support-agent", "advanced-config")
add_agent_variations("support-agent")

---
## 2. Add Completion Mode Variations
From: `SKILL.md` lines 134-214

In [ ]:
def add_completion_variations(config_key: str):
    """Add variations to an existing completion mode AI Config."""
    variations = [
        {
            "key": "creative",
            "name": "Creative Style",
            "messages": [
                {"role": "system", "content": "You are a creative content writer for {{brand}}."},
                {"role": "user", "content": "{{content_request}}"}
            ],
            "model": {
                "modelName": "gpt-4",
                "parameters": {
                    "temperature": 0.9,
                    "maxTokens": 2000,
                    # Optional: custom parameters
                    "style_guide": "creative"
                }
            },
            # Optional: tools created via aiconfig-tools skill
            "tools": [
                {"key": "search_knowledge_base", "version": 1},
                {"key": "get_customer_info", "version": 1}
            ]
        },
        {
            "key": "professional",
            "name": "Professional Style",
            "messages": [
                {"role": "system", "content": "You are a professional content strategist for {{brand}}."},
                {"role": "user", "content": "{{content_request}}"}
            ],
            "model": {
                "modelName": "gpt-4-turbo",
                "parameters": {
                    "temperature": 0.3,
                    "maxTokens": 3000,
                    "style_guide": "professional"
                }
            },
            "tools": [
                {"key": "search_knowledge_base", "version": 1},
                {"key": "get_customer_info", "version": 1}
            ]
        }
    ]

    # Add each variation
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{config_key}/variations"
    headers = {
        "Authorization": API_TOKEN,
        "Content-Type": "application/json",
        "LD-API-Version": "beta"
    }

    created_variations = []

    for variation in variations:
        response = requests.post(url, headers=headers, json=variation)

        if response.status_code in [200, 201]:
            print(f"  [OK] Added variation: {variation['key']}")
            created_variations.append(variation['key'])
            time.sleep(0.5)
        else:
            print(f"  [ERROR] Failed to add variation {variation['key']}: {response.text}")

    if created_variations:
        print(f"\n[OK] Added {len(created_variations)} variations to '{config_key}'")
        print(f"  URL: https://app.launchdarkly.com/projects/{PROJECT_KEY}/ai-configs/{config_key}")
        return True

    return False

# Clean up first, then add variations
print("=== Adding Completion Mode Variations to 'content-assistant' ===\n")
delete_variation("content-assistant", "creative")
delete_variation("content-assistant", "professional")
add_completion_variations("content-assistant")

---
## 3. Get a Specific Variation
From: `SKILL.md` lines 229-260

In [6]:
def get_variation(config_key: str, variation_key: str):
    """Retrieve a specific variation by key."""
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{config_key}"
    headers = {"Authorization": API_TOKEN}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        config = response.json()
        for variation in config.get('variations', []):
            if variation.get('key') == variation_key:
                print(f"[VARIATION] {variation['key']}")
                print(f"  Name: {variation.get('name', 'N/A')}")
                if 'instructions' in variation:
                    print(f"  Mode: Agent")
                elif 'messages' in variation:
                    print(f"  Mode: Completion ({len(variation['messages'])} messages)")
                if 'model' in variation:
                    print(f"  Model: {variation['model'].get('modelName', 'N/A')}")
                if 'tools' in variation and variation['tools']:
                    print(f"  Tools: {[t['key'] for t in variation['tools']]}")
                return variation
        print(f"[ERROR] Variation '{variation_key}' not found")
        return None
    else:
        print(f"[ERROR] Failed to get config: {response.text}")
        return None

# Test get_variation on both configs
print("=== Testing get_variation ===\n")
get_variation("support-agent", "base-config")
print()
get_variation("content-assistant", "creative")

=== Testing get_variation ===

[VARIATION] base-config
  Name: Base Configuration
  Mode: Agent
  Model: gpt-4
  Tools: ['get_customer_info', 'search_knowledge_base']

[VARIATION] creative
  Name: Creative Style
  Mode: Completion (2 messages)
  Model: gpt-4
  Tools: ['get_customer_info', 'search_knowledge_base']


{'_id': 'c03458f1-b972-4f12-b496-b8b005db0aef',
 '_links': {'parent': {'href': '/api/v2/projects/support-ai/ai-configs/content-assistant',
   'type': 'application/json'}},
 '_publishedAt': 1770149626510,
 'createdAt': 1770149626510,
 'key': 'creative',
 'messages': [{'content': 'You are a creative content writer for {{brand}}.',
   'role': 'system'},
  {'content': '{{content_request}}', 'role': 'user'}],
 'model': {'custom': {},
  'modelName': 'gpt-4',
  'parameters': {'maxTokens': 2000,
   'style_guide': 'creative',
   'temperature': 0.9}},
 'name': 'Creative Style',
 'state': 'published',
 'tools': [{'key': 'get_customer_info', 'version': 1},
  {'key': 'search_knowledge_base', 'version': 1}],
 'version': 1}

---
## 4. List All Variations
From: `SKILL.md` lines 350-371

In [7]:
def list_variations(config_key: str):
    """List all variations for an AI Config."""
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{config_key}"
    headers = {"Authorization": API_TOKEN}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        config = response.json()
        variations = config.get('variations', [])
        print(f"[VARIATIONS] {len(variations)} variations in '{config_key}':\n")
        for v in variations:
            print(f"  - {v['key']}: {v.get('name', 'N/A')}")
        return variations
    else:
        print(f"[ERROR] Failed to list variations: {response.text}")
        return []

# Test list_variations
print("=== Testing list_variations ===\n")
list_variations("support-agent")
print()
list_variations("content-assistant")

=== Testing list_variations ===

[VARIATIONS] 3 variations in 'support-agent':

  - default: Default Configuration
  - base-config: Base Configuration
  - advanced-config: Advanced Configuration

[VARIATIONS] 3 variations in 'content-assistant':

  - default: Default Configuration
  - creative: Creative Style
  - professional: Professional Style


[{'_id': '641afbd1-b079-471d-a5f1-591664dc03a3',
  '_links': {'parent': {'href': '/api/v2/projects/support-ai/ai-configs/content-assistant',
    'type': 'application/json'}},
  '_publishedAt': 1770145845050,
  'createdAt': 1770145845050,
  'key': 'default',
  'messages': [{'content': 'You are a creative content writer for {{brand}}. Use available tools when needed.',
    'role': 'system'},
   {'content': '{{content_request}}', 'role': 'user'}],
  'model': {'custom': {},
   'modelName': 'gpt-4',
   'parameters': {'custom_settings': {'audience': 'general',
     'tone': 'friendly'},
    'include_sources': True,
    'maxTokens': 2000,
    'output_format': 'markdown',
    'style_guide': 'creative',
    'temperature': 0.9,
    'topP': 0.95}},
  'name': 'Default Configuration',
  'state': 'published',
  'version': 1},
 {'_id': 'c03458f1-b972-4f12-b496-b8b005db0aef',
  '_links': {'parent': {'href': '/api/v2/projects/support-ai/ai-configs/content-assistant',
    'type': 'application/json'}},
  

---
## 5. Update a Variation
From: `SKILL.md` lines 262-291

In [8]:
def update_variation(config_key: str, variation_key: str, updates: dict):
    """Update an existing variation. Only include fields that need updating."""
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{config_key}/variations/{variation_key}"

    payload = {}
    for field in ["name", "description", "instructions", "messages", "model", "tools"]:
        if field in updates:
            payload[field] = updates[field]

    headers = {"Authorization": API_TOKEN, "Content-Type": "application/json"}
    response = requests.patch(url, json=payload, headers=headers)

    if response.status_code == 200:
        print(f"[OK] Updated variation '{variation_key}'")
        return response.json()
    else:
        print(f"[ERROR] Failed to update variation: {response.text}")
        return None

# Test update_variation - toggle temperature between 0.7 and 0.8
print("=== Testing update_variation ===\n")
print("Before update:")
current = get_variation("support-agent", "base-config")

# Toggle temperature
current_temp = current.get('model', {}).get('parameters', {}).get('temperature', 0.7) if current else 0.7
new_temp = 0.8 if current_temp == 0.7 else 0.7

print(f"\n-> Changing temperature from {current_temp} to {new_temp}...")
updates = {
    "model": {
        "modelName": "gpt-4",
        "parameters": {"temperature": new_temp, "maxTokens": 2000}
    }
}
update_variation("support-agent", "base-config", updates)

print("\nAfter update:")
get_variation("support-agent", "base-config")

=== Testing update_variation ===

Before update:
[VARIATION] base-config
  Name: Base Configuration
  Mode: Agent
  Model: gpt-4
  Tools: ['get_customer_info', 'search_knowledge_base']

-> Changing temperature from 0.7 to 0.8...
[OK] Updated variation 'base-config'

After update:
[VARIATION] base-config
  Name: Base Configuration
  Mode: Agent
  Model: gpt-4
  Tools: ['get_customer_info', 'search_knowledge_base']


{'_id': 'efd133ff-f34d-4a92-98f9-31ab0e60d01f',
 '_links': {'parent': {'href': '/api/v2/projects/support-ai/ai-configs/support-agent',
   'type': 'application/json'}},
 '_publishedAt': 1770149622267,
 'createdAt': 1770149632026,
 'instructions': 'You are a helpful customer support agent.\n\nYour responsibilities:\n- Answer customer questions\n- Resolve issues efficiently\n- Maintain a friendly tone\n\nCompany: {{company_name}}\nPriority: {{support_priority}}',
 'key': 'base-config',
 'model': {'custom': {},
  'modelName': 'gpt-4',
  'parameters': {'maxTokens': 2000, 'temperature': 0.8}},
 'name': 'Base Configuration',
 'state': 'published',
 'tools': [{'key': 'get_customer_info', 'version': 1},
  {'key': 'search_knowledge_base', 'version': 1}],
 'version': 2}

---
## 6. Clone a Variation
From: `SKILL.md` lines 314-348

In [9]:
def clone_variation(config_key: str, source_key: str, new_key: str, modifications: dict = None):
    """Clone an existing variation with optional modifications."""
    source = get_variation(config_key, source_key)
    if not source:
        return None

    new_variation = {
        "key": new_key,
        "name": modifications.get("name", f"Clone of {source.get('name', source_key)}") if modifications else f"Clone of {source.get('name', source_key)}",
    }

    # Copy fields from source, allow modifications to override
    for field in ["instructions", "messages", "model", "tools"]:
        if field in source:
            new_variation[field] = modifications.get(field, source[field]) if modifications else source[field]

    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{config_key}/variations"
    headers = {"Authorization": API_TOKEN, "Content-Type": "application/json"}
    response = requests.post(url, headers=headers, json=new_variation)

    if response.status_code in [200, 201]:
        print(f"[OK] Cloned variation '{source_key}' to '{new_key}'")
        return response.json()
    else:
        print(f"[ERROR] Failed to clone variation: {response.text}")
        return None

# Test clone_variation
print("=== Testing clone_variation ===\n")

# Clean up first
delete_variation("support-agent", "base-config-turbo")

# Clone with modifications
print("\nCloning base-config to base-config-turbo with modified model...")
clone_variation("support-agent", "base-config", "base-config-turbo", {
    "name": "Base Config - Turbo",
    "model": {"modelName": "gpt-4-turbo", "parameters": {"temperature": 0.6}}
})

# Verify clone
print("\nVerifying clone:")
get_variation("support-agent", "base-config-turbo")

=== Testing clone_variation ===

[INFO] Variation 'base-config-turbo' does not exist

Cloning base-config to base-config-turbo with modified model...
[VARIATION] base-config
  Name: Base Configuration
  Mode: Agent
  Model: gpt-4
  Tools: ['get_customer_info', 'search_knowledge_base']
[OK] Cloned variation 'base-config' to 'base-config-turbo'

Verifying clone:
[VARIATION] base-config-turbo
  Name: Base Config - Turbo
  Mode: Agent
  Model: gpt-4-turbo
  Tools: ['get_customer_info', 'search_knowledge_base']


{'_id': '55bf2236-28ff-46f2-b238-eef62fc598c0',
 '_links': {'parent': {'href': '/api/v2/projects/support-ai/ai-configs/support-agent',
   'type': 'application/json'}},
 '_publishedAt': 1770149634630,
 'createdAt': 1770149634630,
 'instructions': 'You are a helpful customer support agent.\n\nYour responsibilities:\n- Answer customer questions\n- Resolve issues efficiently\n- Maintain a friendly tone\n\nCompany: {{company_name}}\nPriority: {{support_priority}}',
 'key': 'base-config-turbo',
 'model': {'custom': {},
  'modelName': 'gpt-4-turbo',
  'parameters': {'temperature': 0.6}},
 'name': 'Base Config - Turbo',
 'state': 'published',
 'tools': [{'key': 'get_customer_info', 'version': 1},
  {'key': 'search_knowledge_base', 'version': 1}],
 'version': 1}

---
## 7. Delete a Variation
From: `SKILL.md` lines 293-310

In [10]:
# delete_variation is already defined above in cell-3
# Test it by deleting the cloned variation we just created

print("=== Testing delete_variation ===\n")

print("Before deletion:")
list_variations("support-agent")

print("\nDeleting 'base-config-turbo'...")
delete_variation("support-agent", "base-config-turbo")

print("\nAfter deletion:")
list_variations("support-agent")

=== Testing delete_variation ===

Before deletion:
[VARIATIONS] 4 variations in 'support-agent':

  - default: Default Configuration
  - base-config: Base Configuration
  - advanced-config: Advanced Configuration
  - base-config-turbo: Base Config - Turbo

Deleting 'base-config-turbo'...
[OK] Deleted variation 'base-config-turbo'

After deletion:
[VARIATIONS] 3 variations in 'support-agent':

  - default: Default Configuration
  - base-config: Base Configuration
  - advanced-config: Advanced Configuration


[{'_id': '1e028480-e9b6-4e82-99af-a820b340c7fd',
  '_links': {'parent': {'href': '/api/v2/projects/support-ai/ai-configs/support-agent',
    'type': 'application/json'}},
  '_publishedAt': 1770145843950,
  'createdAt': 1770145843950,
  'instructions': 'You are a helpful customer support agent.\n\nYour responsibilities:\n- Answer customer questions\n- Resolve issues efficiently\n- Maintain a friendly tone\n\nCompany: {{company_name}}\nPriority: {{support_priority}}',
  'key': 'default',
  'model': {'custom': {},
   'modelName': 'gpt-4',
   'parameters': {'custom_param': 'value',
    'enable_feature': True,
    'maxTokens': 2000,
    'temperature': 0.7,
    'workflow_config': {'max_retries': 3, 'timeout': 30}}},
  'name': 'Default Configuration',
  'state': 'published',
  'version': 1},
 {'_id': 'efd133ff-f34d-4a92-98f9-31ab0e60d01f',
  '_links': {'parent': {'href': '/api/v2/projects/support-ai/ai-configs/support-agent',
    'type': 'application/json'}},
  '_publishedAt': 1770149622267,
